In [1]:
!pip install tensorflow tensorflow-addons matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 31.1 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.1
    Uninstalling typeguard-4.4.1:
      Successfully uninstalled typeguard-4.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.4.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, ReLU, LeakyReLU, Input, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm

# Утилита для загрузки данных
def load_images(data_dir, image_size=(20, 20)):
    images = []
    for file_name in tqdm(os.listdir(data_dir)):
        if file_name.endswith(".npy"):
            image = np.load(os.path.join(data_dir, file_name))
            image = tf.image.resize(image, image_size)  # Изменение размера, если нужно
            images.append(image)
    images = np.array(images)
    images = (images - 0.5) * 2  # Нормализация в диапазон [-1, 1]
    return images

# Загрузка данных
simulation_dir = "C:\Users\kamil\Desktop\vyzkumak\data_preparation\processed_simulation_images"
experiment_dir = "C:\Users\kamil\Desktop\vyzkumak\data_preparation\processed_experiment_images"

sim_images = load_images(simulation_dir)
exp_images = load_images(experiment_dir)

# Функция для построения генератора
def build_generator(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(64, kernel_size=4, strides=2, padding="same")(inputs)
    x = LeakyReLU(0.2)(x)

    x = Conv2D(128, kernel_size=4, strides=2, padding="same")(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)

    x = Conv2DTranspose(128, kernel_size=4, strides=2, padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2DTranspose(64, kernel_size=4, strides=2, padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    outputs = Conv2D(1, kernel_size=7, padding="same", activation="tanh")(x)
    return Model(inputs, outputs, name="generator")

# Функция для построения дискриминатора
def build_discriminator(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(64, kernel_size=4, strides=2, padding="same")(inputs)
    x = LeakyReLU(0.2)(x)

    x = Conv2D(128, kernel_size=4, strides=2, padding="same")(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)

    x = Conv2D(1, kernel_size=4, padding="same")(x)
    outputs = tf.keras.activations.sigmoid(x)
    return Model(inputs, outputs, name="discriminator")

# Построение моделей
input_shape = (20, 20, 1)
gen_G = build_generator(input_shape)
gen_F = build_generator(input_shape)
disc_X = build_discriminator(input_shape)
disc_Y = build_discriminator(input_shape)

# Заморозка дискриминаторов для обучения генераторов
disc_X.trainable = False
disc_Y.trainable = False

# Loss функции и оптимизаторы
loss_fn = tf.keras.losses.MeanSquaredError()
cycle_loss_fn = tf.keras.losses.MeanAbsoluteError()
optimizer = Adam(learning_rate=0.0002, beta_1=0.5)

# Компиляция моделей
gen_G.compile(optimizer=optimizer, loss=loss_fn)
gen_F.compile(optimizer=optimizer, loss=loss_fn)
disc_X.compile(optimizer=optimizer, loss=loss_fn)
disc_Y.compile(optimizer=optimizer, loss=loss_fn)

# Обучение
batch_size = 16
epochs = 100
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    for i in range(0, len(sim_images), batch_size):
        sim_batch = sim_images[i:i+batch_size]
        exp_batch = exp_images[i:i+batch_size]

        # Генерация и обучение
        fake_exp = gen_G.predict(sim_batch)
        fake_sim = gen_F.predict(exp_batch)

        # Обновление дискриминаторов
        disc_X.train_on_batch(sim_batch, fake_sim)
        disc_Y.train_on_batch(exp_batch, fake_exp)

        # Обновление генераторов (Cycle Consistency Loss)
        cycle_sim = gen_F.predict(fake_exp)
        cycle_exp = gen_G.predict(fake_sim)
        gen_G.train_on_batch(sim_batch, cycle_loss_fn(exp_batch, cycle_exp))
        gen_F.train_on_batch(exp_batch, cycle_loss_fn(sim_batch, cycle_sim))

    # Визуализация прогресса
    if epoch % 10 == 0:
        example_sim = sim_images[0:1]
        example_generated = gen_G.predict(example_sim)[0]
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.title("Simulation Input")
        plt.imshow(example_sim[0].squeeze(), cmap="viridis")
        plt.subplot(1, 2, 2)
        plt.title("Generated Experiment")
        plt.imshow(example_generated.squeeze(), cmap="viridis")
        plt.show()


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-2-386896e655aa>, line 24)

In [3]:
from google.colab import files
uploaded = files.upload()